**Glass classification model**

In [ ]:
#downloading weights and biasis
%%capture
!pip install wandb -qqq

In [ ]:
import tensorflow.keras as keras
from google.colab import drive
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback

#constent things I will use in the program
MODELS_PATH = "/content/gdrive/My Drive/magshimim/camerona/models/"
BASE = "glass_base_model.h5"
BEST = "glass_best_model.h5"
DATASET_PATH_DRIVE = "/content/gdrive/My Drive/magshimim/camerona/dataset/glass/"
DATASET_PATH_COLAB = "/content/glass/"

VAL_TEXT = DATASET_PATH_COLAB+"val.txt"
TRAIN_TEXT = DATASET_PATH_COLAB+"train.txt"
TEST_TEXT = DATASET_PATH_COLAB+"test.txt"
#mounting google drive to access files
drive.mount('/content/gdrive/')

In [ ]:
#doing this because in this way the data loads faster
!ln -s "/content/gdrive/My Drive/magshimim/camerona/dataset/glass/" "/content/"

In [ ]:
#loging into the weughts and biasis account
!wandb login

In [ ]:
def buildModel(lr = 0.001):
  """
  this function build the model
  input: lr - learning rate of the model
  output: the model
  """
  input_layer = keras.layers.Input(shape=[64,64,3])

  #block1
  layer = keras.layers.Conv2D(32, 2, strides=1, activation='relu', padding='same', name='block1_conv1')(input_layer)
  layer = keras.layers.MaxPool2D(2, name='block1_max1')(layer)

  #block2
  layer = keras.layers.Conv2D(32, 2, strides=1, activation='relu', padding='same', name='block2_conv1')(layer)
  layer = keras.layers.MaxPool2D(2, name='block2_max1')(layer)

  #output block
  layer = keras.layers.Flatten()(layer)
  layer = keras.layers.Dense(16, activation="relu")(layer)
  layer = keras.layers.Dense(3, activation="softmax", name='output_layer')(layer)
 
  #building and compiling model
  model = keras.Model(inputs=input_layer, outputs=layer)
  opt = keras.optimizers.Adam(learning_rate=lr)
  model.compile(optimizer = opt, loss = ["categorical_crossentropy"], metrics=["categorical_accuracy"])
  print(model.summary())

  return model

In [ ]:
def getData(file_path):
  """
  This function build the data from a file
  input: file path
  output: the x and y of the data
  """
  #openning the data file
  f = open(file_path, 'r')
  data = f.read().split('\n')
  f.close()
  x = []
  y = []
  all = len(data)
  cnt = 1
  #iterating through the file lines
  for line in data:
    if cnt%250 == 0:
      print("{0}/{1}".format(cnt, all))
    cnt += 1
    line_data = line.split(" ")
    try:
      #loading the data image
      img = cv2.imread(DATASET_PATH_COLAB+line_data[0])
      img = cv2.resize(img, (64,64))/255
    except:
      continue
    #adding the y data
    _y = np.zeros(3)
    _y[int(line_data[1])] = 1
    x.append(img)
    y.append(_y)
  #converting to numpy array
  x = np.array(x)
  y = np.array(y)
  return x,y

In [ ]:
wandb.init(project="glass-classification")

In [ ]:
#callbacks
best_callback = keras.callbacks.ModelCheckpoint(MODELS_PATH+BEST, save_best_only=True)
wandb_callback = WandbCallback()

In [ ]:
#loading data
print("loading validation data")
val_x, val_y = getData(VAL_TEXT)
print("loading train data")
train_x, train_y = getData(TRAIN_TEXT)

In [ ]:
#building and saving base model - first time training
model = buildModel()
model.save(MODELS_PATH+BASE)

In [ ]:
model.fit(train_x,
          train_y,
          epochs=24, 
          validation_data=(val_x, val_y),
          callbacks=[best_callback, wandb_callback])

In [ ]:
#loading the best model, we used the tf callback to save it
model = keras.models.load_model(MODELS_PATH+BEST)

In [ ]:
#loading test x and y data
test_x, test_y = getData(TEST_TEXT)

In [ ]:
#evaluating the model
loss, acc = model.evaluate(x = test_x, y = test_y)